# NLP[0] -- 语料和数据收集 -- 文本预处理及数据库文本分析

## 做一个人工智能项目必须得有足够的数据. MutBot这个项目要包含自动收集数据的功能, 打算先从基本的自然语言处理做起...  

本文涉及的内容:  
1. 用第三方库: `langid`检测输入语种, `jieba`做分词和词性标注  
2. 正则表达式re模块的简单运用  
3. SQLite基础数据库操作(创建/检查/写入和读取)
4. 第三方库`pandas`的介绍和简单运用(本例用于简化数据查询操作)

In [1]:
import time # 用于计时比较
import sys  # 本文中用于观察变量内存占用

在做文本处理之前得看看文本到底是不是有意义的语言, 如果是, 不同语言也应该采用不同的方式对待.  
目前主流的开源自然语言语种识别工具有langdetect和langid, 其中, langdetect处理速度较高但准确率不足, langid准确率要高一些但速度也慢.  
langid目前(Jul 15, 2017)提供了97种语言的预训练, 分类出来的的语言标号依据[ISO 639-1](https://baike.baidu.com/item/ISO%20639-1)语言编码标准  

In [2]:
import langid
项目中常见自然语言语种 = {'en':'英文','zh':'中文', 'de':'德语', 'el':'希腊语', 'ja':'日语', 'la':'拉丁语', 'ru':'俄语', 'th':'泰语'}

jieba是一个开源的汉语分词工具, 号称做最好的python中文分词, 简便易用, 支持三种分词模式, 支持繁体, 支持[自定义词典](https://github.com/fxsjy/jieba#%E8%BD%BD%E5%85%A5%E8%AF%8D%E5%85%B8), 可以标注词性, 可以[提取文本关键词](https://github.com/fxsjy/jieba#%E5%9F%BA%E4%BA%8E-tf-idf-%E7%AE%97%E6%B3%95%E7%9A%84%E5%85%B3%E9%94%AE%E8%AF%8D%E6%8A%BD%E5%8F%96). MIT授权.    
jieba只支持中文, 英语以及类英语的语言可以用[NLTK](https://github.com/nltk/nltk)(NLTK也支持中文处理, 但使用不太友好, 中文分词还需要安装斯坦福分词器)  
jieba词性标注采用[和ictclas兼容的标记法](https://gist.github.com/luw2007/6016931)  
其他常用的汉语分词工具还有SnowNLP, PkuSeg, THULAC, HanLP 

In [3]:
计时_单元格开始 = time.time()
import jieba as jb, jieba.analyse as jban, jieba.posseg as pseg
汉语文本常见词性标注表 = {'d':'副词', 'vn':'动名词', 'n':'名词', 'v':'动词', 'a':'形容词', 'y':'语气词', 'c':'连词', 'x':'非语素', 'p':'介词', 'm':'数词', 'q':'量词', 'ul':'助词', 's':'处所词', 'f':'方位词', 'i':'成语', 'ns':'地名', 'o':'拟声词', 'b':'区别词', 'p':'介词', 'r':'代词', 't':'时间词', 'nr':'人名', 'l':'习用语', 'eng':'外来词', 'nz':'其他专名', 'z':'状态词'}
# 有的词性的词对于NLP来说是不重要的, 需要忽略, 比如非语素(标点符号什么的)以及语气词
不关注的词性 = ['x', 'y']
# jieba预设的词典虽然在大多时候能满足需求, 但是对于实际应用来说不够, 很多术语、“黑话”、网络口头语、“梗”会被错判, 还需要根据项目实际情况自制词典.  
# 为了方便观察, 就把自定义词典内容写这里了...
文件_自定字典 = open('userDict.txt', 'w', -1)
文件_自定字典.write('''
剪脚封灌 30 vn
感应加热 600 nz
市电 200 d
那 10 c
上去 20 z
上去过 6000 vf
下去 20 z
靠 10 y
是不 5000 l
初级 20 nz
次级 20 nz
斩波 200 nz
TC 600 nz
发错 10 z
群 200 n
控制 300 vn
可行 20 z
没有卖的 z
啊 6 y
地 5000 nz
良好接地 4960 z
异常 20 nz
指定的 300 d
raise语句 nz
except语句 nz
新洁能 nz
可行 40 z
''')
文件_自定字典.flush(); 文件_自定字典.close()
jb.load_userdict('userDict.txt')
jb.initialize()
处理耗时 = (time.time() - 计时_单元格开始) * 1000
print("debug --- 处理本单元格耗时 %0.3f 毫秒" % 处理耗时 )

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.666 seconds.
Prefix dict has been built successfully.


debug --- 处理本单元格耗时 4033.452 毫秒


In [4]:
测试文本1 = "[CQ:at,id=qq/user/15*****] 😂氖泡接在驱动电源负和地线之间[CQ:face,id=108][CQ:face,id=108][CQ:at,id=qq/user/15*****]"
测试文本2 = "牛啤"
测试文本3 = "ΕΞΖΝи△M"
测试文本4 = "This is a test text..."
测试文本5 = '<h2><a id="user-content-probability-normalization" class="anchor" aria-hidden="true" href="#probability-normalization"></a>Probability Normalization</h2>'
测试文本6 = "import win32event as event; event_tigger = event.CreateEvent(None, False, False, 'Global\\233__update'); event.SetEvent(event_tigger)"

In [5]:
langid.classify(""); time_start = time.time()
print( langid.classify(测试文本1), langid.classify(测试文本2), langid.classify(测试文本3), langid.classify(测试文本4), langid.classify(测试文本5), langid.classify(测试文本6) ); 
print("语种检测(常规模式)耗时→%.3f秒" % ( time.time() - time_start ))
# langid.classify默认输出的第二个数值是对数概率, 不计算全部语种的概率所以速度快. 但是有时候因为还需要知道语种分类的"置信度", 就需要启用langid.py的概率归一化: 
time_start = time.time()
from langid.langid import LanguageIdentifier, model
lider = LanguageIdentifier.from_modelstring(model, norm_probs=True) # 其实主要的耗时是出在这里
print("启用置信度归一化耗时→%.3f秒" % ( time.time() - time_start ))
time_start = time.time()
print( lider.classify(测试文本1), lider.classify(测试文本2), lider.classify(测试文本3), lider.classify(测试文本4), lider.classify(测试文本5), lider.classify(测试文本6) ); 
print("语种检测(归一化置信度模式)耗时→%.3f秒" % ( time.time() - time_start ))

('zh', -204.03105926513672) ('zh', -19.84054446220398) ('el', -68.03624391555786) ('en', -54.41310358047485) ('eu', -78.12822103500366) ('en', -47.138615131378174)
语种检测(常规模式)耗时→0.026秒
启用置信度归一化耗时→13.849秒
('zh', 1.0) ('zh', 0.5123629134775305) ('el', 0.9340167392870947) ('en', 0.999999990990354) ('eu', 0.5994969697756787) ('en', 0.998501267059482)
语种检测(归一化置信度模式)耗时→0.030秒


还可以考虑加个编程语言探测, 比如GitHub通过[linguist](https://github.com/github/linguist)对主流编程语言(300多种)达到84%正确识别   
其实也可以自己用朴素贝叶斯实现一个, 但是暂时没时间和精力去做了...

In [6]:
def 语种测试(文本):
    语种 = ""
    语种分类信息 = lider.classify(文本)
    if 语种分类信息[1] > 0.98: # 实际观察发现, 置信度低于这个值的通常是无意义内容或者不是自然语言
        try:
            语种 = 项目中常见自然语言语种[语种分类信息[0]]
        except KeyError:
            语种 = "未知"
    else:
        语种 = None
    return 语种

def 分词并标注(文本, 调试模式 = False):
    分词结果 = []; 标注结果 = []
    文本的语种 = 语种测试(文本)
    if 文本的语种 != None:
        if 调试模式 == True:
            print("###debug---检测到 %s 文本 \"%s\" 输入---" % (文本的语种, 文本) )
        if 文本的语种 == "中文":
            tmp_pseg结果 = pseg.lcut(文本)
            if 调试模式 == True:
                结果 = []
                for 词, 词性 in tmp_pseg结果:
                    try:
                        结果.append( (词, "%s(%s)" % ( 词性, 汉语文本常见词性标注表[词性]) ) )
                    except KeyError:
                        结果.append( (词, 词性) )
                return 结果
            else:
                for 词, 词性 in tmp_pseg结果:
                    if 词性 not in 不关注的词性: # 忽略多余的语素
                        分词结果.append( 词 )
                        标注结果.append( 词性 )
        else:
            raise UserWarning("暂不处理其他语种")
    else:
         raise SyntaxWarning("无法确定输入文本的语种")
    return 分词结果, 标注结果
print(分词并标注("Python可以使用raise语句抛出一个指定的异常, 之后再使用except语句根据异常信息来处理.", 调试模式 = True))

###debug---检测到 中文 文本 "Python可以使用raise语句抛出一个指定的异常, 之后再使用except语句根据异常信息来处理." 输入---
[('Python', 'eng(外来词)'), ('可以', 'c(连词)'), ('使用', 'v(动词)'), ('raise语句', 'nz(其他专名)'), ('抛出', 'v(动词)'), ('一个', 'm(数词)'), ('指定的', 'd(副词)'), ('异常', 'nz(其他专名)'), (',', 'x(非语素)'), (' ', 'x(非语素)'), ('之后', 'f(方位词)'), ('再', 'd(副词)'), ('使用', 'v(动词)'), ('except语句', 'nz(其他专名)'), ('根据', 'p(介词)'), ('异常', 'nz(其他专名)'), ('信息', 'n(名词)'), ('来', 'v(动词)'), ('处理', 'v(动词)'), ('.', 'x(非语素)')]


-----
MutBot这个项目以QQ作为主要数据来源, 所以预处理除了做自然语言语种判断、分词和词性标注之外,  还需要过滤CQ码以及多余的符号. 这里用正则表达式简单处理  

In [7]:
import re
def CQ记录转换(原CQ记录, 调试模式 = False):
    if 调试模式 == True:
        print("***debug--传入文本 \"%s\"" % 原CQ记录)
    消息_处理 = 原CQ记录
    消息 = {"CQ码":"", "内容":""}
    tmp = re.search('(?P<CQ码>\[CQ:\w+,\S+\])', 消息_处理)
    tmpcounter = 0
    while tmp != None and tmpcounter < 5:     # 把CQ码全都单独收起来, 剩下的就是内容了
        消息["CQ码"] += tmp.group(1) + ", "
        消息_处理 = 消息_处理.replace(tmp.group(1), "")
        tmp = re.search('(?P<CQ码>\[CQ:\w+,\S+\])', 消息_处理)
        tmpcounter += 1
    消息["内容"] = 消息_处理
    return 消息

In [8]:
消息 = CQ记录转换(测试文本1, 调试模式 = True); print(消息)

***debug--传入文本 "[CQ:at,id=qq/user/15*****] 😂氖泡接在驱动电源负和地线之间[CQ:face,id=108][CQ:face,id=108][CQ:at,id=qq/user/15*****]"
{'CQ码': '[CQ:at,id=qq/user/15*****], [CQ:face,id=108][CQ:face,id=108], ', '内容': ' 😂氖泡接在驱动电源负和地线之间'}


----  

以上内容基本就能实现简单的文本预处理了, 预处理数据作为后级的输入  
**目前主要的目标还是从QQ群聊天记录里提取语料和数据集**  

----

目前(05 Jan. 2020)的想法: 维护一个数据库用以处理和记录对话上下文关系以及前级(文本预处理等)的结果和处理情况

In [9]:
import sqlite3 as sqlite

In [10]:
import numpy as np, pandas as pd # 主要是为了简化原始数据的读取和预处理
pd.set_option('max_colwidth',100); pd.set_option('display.max_rows', 20); pd.set_option('display.max_columns', 8) # 为了方便查看内容

>  pandas是一款数据处理工具，集成了numpy以及matplotlib，拥有便捷的数据处理以及文件读取能力  

pandas主要有几大功能:  
1. Object Creation: pandas有三种对象, series、df和Panel Object. Series, 通过传入list对象来新建, 可以指定索引; DataFrame, 通过传入numpy数组/dictionary对象来创建.
2. Viewing Data: 查看头部/查看索引和列名/查看统计结果什么的, 还可以转换成numpy格式/做矩阵转置以及排序.
3. Selection: 类似于sql的select, 可以指定列/行/标签/值/位置/条件来筛选
4. Missing Data: 默认的空值是np.nan, 可通过reindex函数来增删改查某坐标轴(行或列)的索引，并返回一个数据的拷贝, 还可以判断是否为空值(返回False或True)
5. Operations: 一些常用计算, 包括平均值/ 数值移动等. (通过应用还可以做累计求和和其他自定义的方法)
6. Merge: pandas提供了多个方法来合并不同的对象. 其中Merge方法类似SQL的合并方式.
7. Grouping: 分组主要是为了对某些数据的计算, 例如df.groupby('A').sum()
8. Reshaping
9. Time Series: pandas很适合用来处理时序, 可以调整时间间隔/时区转换/时间格式转换
10. [Categoricals](http://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html#categorical)
11. Plotting: 用于数据绘图, 和Matplotlib基本一样
12. Getting Data In/Out : 可以方便地在不同类型的文件(csv/text/json/html/excel/sql等)中导入或导出数据

完整的[user guide](https://pandas.pydata.org/pandas-docs/stable/user_guide/)可以在pydata上看到  
*这里主要是要用pandas的DataFrame简化sql操作*  
用 pandas DataFrame 读取数据结果的好处主要是不需要每次都调用 fetchall之类的函数, 还能能方便地通过表头的名字来阅读整个表

In [11]:
DB_CQ = sqlite.connect("eventv2.db")
DB_APP = sqlite.connect("app.db")
# 出于尊重隐私考虑, 不提供这两个数据库, 并且这两个数据库相关的部分内容打了码

In [12]:
计时_单元格开始 = time.time()
query_selectLog = "SELECT `id`, `tag`, `GROUP`, `account`, `operator`, `content`, `TIME` FROM `event` ORDER by `id` DESC;"
DF_CQdata = pd.read_sql_query(query_selectLog, DB_CQ)
处理耗时 = (time.time() - 计时_单元格开始) * 1000
print("debug --- 处理本单元格耗时 %0.3f 毫秒" % 处理耗时 )
print("debug --- 对象`DF_CQdata`占用内存 → %0.3fKB" % (sys.getsizeof(DF_CQdata) / 1000)  )

debug --- 处理本单元格耗时 113.650 毫秒
debug --- 对象`DF_CQdata`占用内存 → 9271.409KB


In [13]:
DF_CQdata

,id,tag,group,account,operator,content,time
0,23030,contact,qq/group/56961***,qq/user/35013***,,"[CQ:image,file=916B138244BF4508EE7A3C17D34696CC.jpg]",1579152878
1,23029,contact,qq/group/56961***,qq/user/35013***,,毕竟他上次更新是在去年,1579152875
2,23028,contact,qq/group/56961***,qq/user/13365***,,差一点我就五级了,1579152873
3,23027,contact,qq/group/56961***,qq/user/35145***,,"[CQ:face,id=182]",1579152866
4,23026,contact,qq/group/56961***,qq/user/13365***,,"[CQ:image,file=F1F1AB4152D45DB9D05D31E38E47E0B5.jpg]",1579152862
...,...,...,...,...,...,...,...
23025,5,,,qq/user/2139223150,,来自616471607的私聊消息(4):,1577880261
23026,4,contact,,qq/user/616471607,,29,1577880261
23027,3,contact,qq/group/56961***,qq/user/15284***,,像我这种小功率都内部接地,1577880243
23028,2,contact,qq/group/56961***,qq/user/51387***,,"[CQ:at,id=qq/user/14262***] [CQ:at,id=qq/user/14262***] 我就谁便用了啵",1577880188


In [14]:
计时_单元格开始 = time.time()
经过预处理的消息 = {} 
tmp_CQ转换 = ""
tmp_分词标注 = ""
time_start = time.time()
for row in DF_CQdata.itertuples(): 
# 用.iterrows()虽然可读性更好, 但是它是真的慢...用.iterrows()的话耗时差不多都是280到360秒...所以还是改用.itertuples()
    tmp_CQ转换 = CQ记录转换(row[6])
    if len(tmp_CQ转换["内容"]) > 0:
        try:
            tmp_分词标注 = 分词并标注(tmp_CQ转换["内容"]) 
        except UserWarning:
            tmp_分词标注 = ["[*unkownLanguage*]" + tmp_CQ转换["内容"], []]
        except SyntaxWarning:
            tmp_分词标注 = ["[*unclearContent*]" + tmp_CQ转换["内容"], []]
    else:
        tmp_分词标注= [[],[]]
    经过预处理的消息[row[1]] = {'CQ码':tmp_CQ转换['CQ码'] if len(tmp_CQ转换['CQ码']) > 0 else "", '分词标注':[tmp_分词标注[0], tmp_分词标注[1]] if len(tmp_分词标注[0]) > 0 else [[], []]}
    
print("文本预处理了 %d 条记录. 耗时→%.3f秒" % (len(经过预处理的消息), time.time() - time_start) )
tmp_对照变量 = "abcde12345"
print("debug --- 变量`tmp_对照变量`占用内存 → %0.3fKB" % (sys.getsizeof(tmp_对照变量) / 1000) )
print("debug --- 变量`经过预处理的消息`占用内存 → %0.3fKB" % (sys.getsizeof(经过预处理的消息) / 1000) )

tmp_循环限制计次 = 0
for 消息ID, 预处理结果 in 经过预处理的消息.items():
    tmp_循环限制计次 += 1
    print("%d: CQ码→%s | 分词标注→%s" %(消息ID, 预处理结果['CQ码'] if len(预处理结果['CQ码']) > 0 else "无", 预处理结果['分词标注'] if len(预处理结果['分词标注'][0]) > 0 else "无" ) )
    if tmp_循环限制计次 > 35: break
处理耗时 = (time.time() - 计时_单元格开始) * 1000
print("debug --- 处理本单元格耗时 %0.3f 毫秒" % 处理耗时 )

文本预处理了 23030 条记录. 耗时→253.659秒
debug --- 变量`tmp_对照变量`占用内存 → 0.074KB
debug --- 变量`经过预处理的消息`占用内存 → 1310.824KB
23030: CQ码→[CQ:image,file=916B138244BF4508EE7A3C17D34696CC.jpg],  | 分词标注→无
23029: CQ码→无 | 分词标注→[['毕竟', '他', '上次', '更新', '是', '在', '去年'], ['d', 'r', 't', 'd', 'v', 'p', 't']]
23028: CQ码→无 | 分词标注→[['差一点', '我', '就', '五级', '了'], ['d', 'r', 'd', 'b', 'ul']]
23027: CQ码→[CQ:face,id=182],  | 分词标注→无
23026: CQ码→[CQ:image,file=F1F1AB4152D45DB9D05D31E38E47E0B5.jpg],  | 分词标注→无
23025: CQ码→[CQ:image,file=08CD688B8A40ADDDAE11A02D26555B84.jpg],  | 分词标注→无
23024: CQ码→无 | 分词标注→[['没事', '这', '是', '此', 'UP', '的', '视频', '开头', '的', '独特性'], ['v', 'r', 'v', 'r', 'eng', 'uj', 'n', 'v', 'uj', 'b']]
23023: CQ码→无 | 分词标注→[['测试', '202001161332'], ['vn', 'm']]
23022: CQ码→无 | 分词标注→[['来自', '616471607', '的', '私聊', '消息', '23021'], ['v', 'm', 'uj', 'a', 'n', 'm']]
23021: CQ码→无 | 分词标注→[['测试', '202001161332'], ['vn', 'm']]
23020: CQ码→无 | 分词标注→[['测试', '202001161329'], ['vn', 'm']]
23019: CQ码→无 | 分词标注→[['来自', '616471607', 

In [15]:
# 再维护一个数据库用以记录预处理过的内容以及上下文关系
DB_ppced = r"CQevePPC.db"
Conn_ppced = sqlite.connect(DB_ppced)

目前(10 Jan. 2020)的想法是使用两个表: `ppcLog` 和 `contextIndex`, 分别用于记录预处理后的数据和上下文的关系(按群和时间分组, 以便后续提取对话)

In [16]:
# 数据库设计
query_createTable_ppcLog = '''
CREATE TABLE "ppcLog" (
"lid"       INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
"Link"      INTEGER,
"cqCode"    TEXT,
"jiebaCUT"  TEXT,
"jiebaPSEG" TEXT,
"lastModif" NUMERIC NOT NULL DEFAULT CURRENT_TIMESTAMP,
"note"      TEXT
);'''
query_createTable_contextIndex = '''
CREATE TABLE "contextIndex" (
"id_conversation"   INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
"GroupFrom"         INTEGER,
"UserFrom"          INTEGER,
"dialogsContent"    TEXT,
"DialogsLink"       INTEGER
"time_sort"         NUMERIC NOT NULL DEFAULT CURRENT_TIMESTAMP,
"note"              TEXT
);'''

In [17]:
# 先检查数据库状态, 如果没有预设的表那就初始化
def DBtest(dbConnHandle, tableNames, checkQuery = []):
    querys_tableCheck = []
    for i in range( len(tableNames) ):
        querys_tableCheck.append("SELECT * FROM `sqlite_master` where type = 'table' and name = '%s';" % tableNames[i])
    DBisOK = False
    OrigSct = ""
    tmp_cont = 0
    cont_table = 0
    cont_unexc = 0
    for query in querys_tableCheck:
        curs = dbConnHandle.execute(query)
        gets = curs.fetchall()
        if len(gets) > 0:
            OrigSct = gets[-1][-1]
            print("数据表`%s`存在. ~ 读到记录:  \n%s " % (tableNames[tmp_cont], OrigSct) )
            cont_table += 1
            if len(checkQuery) == len(tableNames):
                if OrigSct.replace("\n", "").replace(" ","").replace(";", "").replace("\t", "") != checkQuery[tmp_cont].replace("\n", "").replace(" ","").replace(";", "").replace("\t", ""):
                    cont_unexc -= 4
                else:
                    DBisOK += 1
        tmp_cont += 1

    if checkQuery != []:
        if cont_table < 1 :
            print("指定的表不存在")
            DBisOK = False
        if cont_unexc < 0: 
            if cont_table > 0:
                raise SyntaxWarning("数据库检查未通过: 已经存在不同结构的同名表. (有 %d 个表记录与提供的记录不一致)" % (-cont_unexc / 4))
                return cont_unexc
        else:
            if DBisOK == len(tableNames) :
                print("数据库检查通过. %d 个表(共%d个表)校验一致" % (DBisOK, len(tableNames)) )
                DBisOK = True    
    else:
        if DBisOK == len(tableNames) :
            print("指定表存在")
            DBisOK = True
        else:
            print("指定表不存在")
            DBisOK = False
    return DBisOK        

In [18]:
def DB_init(dbConnHandle, tableNames = [], checkQuery = []):
    realyNeedInit = False
    try:
        dbStatus = DBtest(dbConnHandle, tableNames, checkQuery)
    except SyntaxWarning as info:
        print("初始化失败, 建议检查运行环境(文件冲突). 错误信息:  \n    ", info)
        return -233
    if dbStatus == False:
        realyNeedInit = True
    elif dbStatus < 0:
        r
    if realyNeedInit == True:
        print("准备开始初始化...")
        try:
            for query in checkQuery:
                dbConnHandle.execute(query)
            print("提交变动...")
            dbConnHandle.commit()
            print("数据表初始化完成")
        except Exception as errinfo: 
            print("---SQL执行失败---: ", errinfo)
    else:
        print("已存在同名表, 跳过初始化")

In [19]:
DB_init(Conn_ppced, ["ppcLog", "contextIndex"], [query_createTable_ppcLog, query_createTable_contextIndex])

指定的表不存在
准备开始初始化...
提交变动...
数据表初始化完成


In [20]:
DBtest(Conn_ppced, ["ppcLog", "contextIndex"], [query_createTable_ppcLog, query_createTable_contextIndex])

数据表`ppcLog`存在. ~ 读到记录:  
CREATE TABLE "ppcLog" (
"lid"       INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
"Link"      INTEGER,
"cqCode"    TEXT,
"jiebaCUT"  TEXT,
"jiebaPSEG" TEXT,
"lastModif" NUMERIC NOT NULL DEFAULT CURRENT_TIMESTAMP,
"note"      TEXT
) 
数据表`contextIndex`存在. ~ 读到记录:  
CREATE TABLE "contextIndex" (
"id_conversation"   INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
"GroupFrom"         INTEGER,
"UserFrom"          INTEGER,
"dialogsContent"    TEXT,
"DialogsLink"       INTEGER
"time_sort"         NUMERIC NOT NULL DEFAULT CURRENT_TIMESTAMP,
"note"              TEXT
) 
数据库检查通过. 2 个表(共2个表)校验一致


True

In [21]:
# 接下来是把预处理好的内容存入数据库, 并标注哪些内容已经处理以及修改时间

# 为了提高插入数据的效率, 使用"executemany". 当然, 还可以用pandas.DataFrame.to_sql, 不过这里就怎么省事怎么来了.  
# 需要注意的是, 如果想要用sqlite3的python接口"executemany", 传入数据需要是`[(xxx,xx,...,x),...(x,xxx,...,xx)]`的结构, 其中每个元组的元素数要对应占位符. 另外, 一次处理的量不能太大.
  
Conn_ppced.executemany(
    "insert into ppcLog (`Link`, `cqCode`, `jiebaCUT`, `jiebaPSEG`, `note`) values (?, ?, ?, ?, 'initialize')",
    [ ( link, res['CQ码'], str(res["分词标注"][0]), str(res["分词标注"][1]) )  for link, res in 经过预处理的消息.items() ]
)
Conn_ppced.commit() # 操作后应该尽快提交, 以免数据库锁带来的麻烦(尤其是多线程)

In [22]:
# 提交完了, 瞅瞅情况
query_selectLog = "SELECT * FROM `ppcLog`;"
数据库中的预处理结果 = pd.read_sql_query(query_selectLog, Conn_ppced)

In [23]:
数据库中的预处理结果

,lid,Link,cqCode,jiebaCUT,jiebaPSEG,lastModif,note
0,1,23030,"[CQ:image,file=916B138244BF4508EE7A3C17D34696CC.jpg],",[],[],2020-01-27 14:48:59,initialize
1,2,23029,,"['毕竟', '他', '上次', '更新', '是', '在', '去年']","['d', 'r', 't', 'd', 'v', 'p', 't']",2020-01-27 14:48:59,initialize
2,3,23028,,"['差一点', '我', '就', '五级', '了']","['d', 'r', 'd', 'b', 'ul']",2020-01-27 14:48:59,initialize
3,4,23027,"[CQ:face,id=182],",[],[],2020-01-27 14:48:59,initialize
4,5,23026,"[CQ:image,file=F1F1AB4152D45DB9D05D31E38E47E0B5.jpg],",[],[],2020-01-27 14:48:59,initialize
...,...,...,...,...,...,...,...
23025,23026,5,,"['来自', '616471607', '的', '私聊', '消息']","['v', 'm', 'uj', 'a', 'n']",2020-01-27 14:49:00,initialize
23026,23027,4,,[*unclearContent*]29,[],2020-01-27 14:49:00,initialize
23027,23028,3,,"['像', '我', '这种', '小', '功率', '都', '内部', '接地']","['v', 'r', 'r', 'a', 'n', 'd', 'f', 'v']",2020-01-27 14:49:00,initialize
23028,23029,2,"[CQ:at,id=qq/user/14262***],","['我', '就', '谁', '便用', '了']","['r', 'd', 'r', 'v', 'ul']",2020-01-27 14:49:00,initialize


-----

### 以上内容便实现了数据接口以及数据预处理的基础功能(所以作为第0部分), 接下来开始真正的NLP内容  

本文内容仅为个人见解和心得, 希望能带来帮助. 如果有不正确/不准确之处还请多多指教, 一起学习一起进步~   

本文内容由[佚之狗](https://github.com/HookeLiu)原创, 可以随意使用但请注明出处.    